In [7]:
from datetime import datetime
import hopsworks

from src.data import *
from src.config import *

#### 1. Download Raw data

In [3]:
%%time

from_year = 2022
to_year = datetime.now().year
  
#Download all raw files   
for year in range(from_year,to_year+1):
    download_files_raw(year)
print("Downloaded raw files from",from_year,"to",to_year)

Downloaded files :  ['rides_2022-01.parquet', 'rides_2022-02.parquet', 'rides_2022-03.parquet', 'rides_2022-04.parquet', 'rides_2022-05.parquet', 'rides_2022-06.parquet', 'rides_2022-07.parquet', 'rides_2022-08.parquet', 'rides_2022-09.parquet', 'rides_2022-10.parquet', 'rides_2022-11.parquet', 'rides_2022-12.parquet']
Downloaded files :  ['rides_2023-01.parquet', 'rides_2023-02.parquet', 'rides_2023-03.parquet', 'rides_2023-04.parquet', 'rides_2023-05.parquet', 'rides_2023-06.parquet', 'rides_2023-07.parquet', 'rides_2023-08.parquet', 'rides_2023-09.parquet', 'rides_2023-10.parquet', 'rides_2023-11.parquet']
Downloaded files :  []
Downloaded raw files from 2022 to 2024
CPU times: user 8.43 s, sys: 4.47 s, total: 12.9 s
Wall time: 1min 36s


#### 2. Validate raw data

In [4]:
%%time

validate_data_files()

CPU times: user 43.9 s, sys: 14.7 s, total: 58.6 s
Wall time: 38.4 s


,pickup_time,pickup_location
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68
...,...,...
74587601,2023-11-30 23:39:21,161
74587602,2023-11-30 23:01:55,232
74587603,2023-11-30 23:23:16,113
74587604,2023-11-30 23:39:22,90


#### 3. Transform the validated data into Time Series data

In [5]:
%%time

taxi_data_ts = transform_to_timeseries()
taxi_data_ts.columns = ['pickup_hour','pickup_location_id','rides']
taxi_data_ts

CPU times: user 2min 4s, sys: 1min 45s, total: 3min 49s
Wall time: 3min 55s


,pickup_hour,pickup_location_id,rides
0,2022-01-01 00:00:00,87,32
1,2022-01-01 00:00:00,237,181
2,2022-01-01 00:00:00,41,12
3,2022-01-01 00:00:00,265,4
4,2022-01-01 00:00:00,148,104
...,...,...,...
4395307,2023-11-30 23:00:00,253,0
4395308,2023-11-30 23:00:00,254,0
4395309,2023-11-30 23:00:00,257,0
4395310,2023-11-30 23:00:00,258,0


#### 4. Hopsworks feature store

In [8]:
### Connecting to Hopsworks Feature Store


hw_project = hopsworks.login(project=HOPSWORKS_PROJECT,api_key_value=HOPSWORKS_API_KEY)
fs = hw_project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/467093
Connected. Call `.close()` to terminate connection gracefully.


In [9]:
### Create Feature Group

fg = fs.get_or_create_feature_group(name=FEATURE_GROUP_NAME,
                             description='Hourly data of taxi demand in NYC in TimeSeries format',
                             version=FEATURE_GROUP_VERSION,
                             primary_key=['pickup_hour','pickup_location_id'],
                             event_time='pickup_hour')

In [10]:
%%time

### Insert data into Feature Group and wait till it finishes

fg.insert(taxi_data_ts)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/467093/fs/462916/fg/529750


Uploading Dataframe: 0.00% |          | Rows 0/4395312 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: taxi_time_series_hourly_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/467093/jobs/named/taxi_time_series_hourly_1_offline_fg_materialization/executions
CPU times: user 3min 5s, sys: 15.2 s, total: 3min 20s
Wall time: 3min 50s


(<hsfs.core.job.Job at 0x132330940>, None)